In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Perceptron stuff

In [ ]:
# AND gate
X = np.array([
    [1, 0, 0],
    [1, 0, 1],
    [1, 1, 0],
    [1, 1, 1],
])

# AND
y = np.array([0, 0, 0, 1], dtype=np.int8)

# OR
# y = np.array([0, 1, 1, 1], dtype=np.int8)

# XOR
# y = np.array([0, 1, 1, 0], dtype=np.int8)

In [ ]:
# plotting stuff
def plot_inputs(X, y):
    y_flat = y.flatten()
    fig, ax = plt.subplots(1, 1)
    ax.scatter(X[y_flat == 0,1],X[y_flat == 0,2],color="blue", marker="o", label="Output: 0")
    ax.scatter(X[y_flat == 1,1],X[y_flat == 1,2],color="red", marker="x", label="Output: 1")
    ax.set_xlabel("$x_1$")
    ax.set_ylabel("$x_2$")
    ax.set_aspect("equal")
    return fig, ax

def plot_decision(ax, w, colour=[0, 0, 0, 0.2], label=None):
    # We get a 1 if w0 + w1x1 + w2x2 >= 0, and a 0 otherwise.
    # solving for x2, we get x2 >= -(w0 + w1x1) / w2
    if abs(w[2]) > 0:
        slope = -w[1] / w[2]
        intercept = -w[0] / w[2]
        decision_x = np.array([-0.1, 1.1])
        ax.plot(decision_x, decision_x * slope + intercept, color=colour, label=label)

fig, ax = plot_inputs(X, y)
leg = ax.legend()

In [ ]:
eta = 1 # early choice, gives integer weights

def step(v):
    """Heaviside step function"""
    return (v >= 0).astype(int)

# Initialize weights to zero
w = np.zeros(3)

y_hat = X @ w
iter = 0

while not all(y == y_hat):
    # Visualize progress
    plot_decision(ax, w)
    print(f"Iteration {iter}: w = {w}")
    
    # one instance at a time
    i = iter % 4
    y_hat[i] = step(w.T @ X[i,:])
    if y_hat[i] != y[i]:
        w += eta * (y[i] - y_hat[i]) * X[i,:]
    iter += 1

print(f"Final w = {w.flatten()}")

plot_decision(ax, w, "green", "Final boundary")
ax.set_aspect("auto")
leg.remove()
leg = ax.legend()
ax.set_title(f"Solution converged after {iter} iterations")
fig


In [ ]:
# Verify XOR solution
# X is unchanged from prior inputs
y_xor = np.array([0, 1, 1, 0])
w_1 = np.array([
    [-3/2, -1/2],
    [1, 1],
    [1, 1],
])

# intermediate "hidden" space - inputs to the next layer
H = step(X @ w_1)

# add on the bias term for the hidden layer
H = np.column_stack((np.ones(4), H))
w_2 = np.array([-0.5, -1, 1])

fig_h, ax_h = plot_inputs(H, y_xor)
plot_decision(ax_h, w_2, label="Decision boundary for layer 2")
ax_h.set_aspect("auto")
ax_h.set_xlabel("$h_1$")
ax_h.set_ylabel("$h_2$")
ax_h.legend()

print(step(H @ w_2))

## Backpropagation examples

In [ ]:
# toy mlp example
# forward pass
X = np.array([2, 3])
y = 1
W1 = np.array([[-0.78, 0.13], [0.85, 0.23]])
W2 = np.array([1.8, 0.40])


In [ ]:
X @ W1 @ W2

In [ ]:

iterations = 20
eta = 0.01
loss = np.zeros(iterations)

for i in range(iterations):
    # forward pass
    y_hat = X @ W1 @ W2
    
    # update loss to keep track of performance
    loss[i] = (y_hat - y)**2

    # backpropagate!
    w2_partials = (y_hat - y) * (X @ W1)
    w1_partials = w2_partials @ X

    # take a step in the opposite direction
    W1 = W1 - eta * w1_partials
    W2 = W2 - eta * w2_partials

plt.plot(loss)

# check how well we did
print("Final prediction:", X @ W1 @ W2)